## Packages

From termainal I did:

```
pipenv install
pipenv install jupyter pymysql sqlalchemy requests
```


In [1]:
import pymysql.cursors
import requests
from datetime import datetime
from sqlalchemy import create_engine
import hashlib 
import os
import json
import time
from collections import defaultdict
from flask import jsonify
import re
import pandas as pd
import s3fs
import boto3

# variables we'll need
host = os.environ['DBHOST']
port = 3306
dbname = "collab"
user = os.environ['DBUSER']
password = os.environ['DBPASSWORD']

In [2]:
#### Main function ####

def handler(incoming):
    
    ## Put code here
    
    
    return True

In [3]:
########

In [4]:
# create sqlalchemy engine
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                   .format(user=user,
                           pw=password,
                           host=host,
                           db=dbname))

In [5]:
# Get the data we need
dbConnection    = engine.connect()

df = pd.read_sql("SELECT `identifier`, `item_key`, `item_value` FROM `data_pieces`", dbConnection)
columns_df = pd.read_sql("SELECT * FROM `column_tracker`", dbConnection)
text_df = pd.read_sql("SELECT * FROM `text_log`", dbConnection)
first_contact_df = pd.read_sql("SELECT * FROM `first_contact`", dbConnection)

dbConnection.close()

In [6]:
pivoted = df.pivot(index='identifier', columns='item_key', values=['item_value'])

In [7]:
# pivoted

In [8]:
pivoted.columns = pivoted.columns.get_level_values(1)
pivoted.reset_index(inplace=True) 

In [9]:
# pivoted

In [10]:
# pivoted.columns

In [11]:
columns_df.sort_values(by=['created_at'], inplace=True)

In [12]:
ordered_columns = columns_df['col'].tolist()

In [13]:
ordered_columns.insert(0, "identifier")

In [14]:
# ordered_columns

In [15]:
pivoted_ordered = pivoted[ordered_columns]

In [16]:
# pivoted_ordered

In [17]:
# text_df

In [18]:
text_concat = text_df.groupby(['identifier'])['raw_text'].apply(' | '.join).reset_index()



In [19]:
merge1 = pd.merge(pivoted_ordered, text_concat, on="identifier")

In [20]:
# merge1

In [21]:
first_contact_df.sort_values(by=['created_at'], inplace=True)

In [22]:
final_table = pd.merge(first_contact_df, merge1, on="identifier")

In [23]:
# final_table

In [24]:
csv_file = f"{dbname}.csv"
final_table.to_csv(csv_file, index=False)

In [25]:
s3 = boto3.client('s3')
s3.upload_file(
    csv_file, 'collab-bridge', csv_file,
    ExtraArgs={'ACL': 'public-read', 'CacheControl':'no-cache'}
)

In [26]:
# final_table.to_csv(f's3://collab-bridge/{dbname}-export-v3.csv', index=False)

In [27]:
### Make it public
#s3 = boto3.resource('s3')
#s3.Object('collab-bridge', f'{dbname}-export.csv').Acl().put(ACL='public-read')


In [28]:
## Note: Testing URL is https://collab-bridge.s3.amazonaws.com/collab-export-v4.csv